## torchtext.data.batch.py
### class Batch(object)
#### 属性
- batch_size: batch的大小
- dataset: 数据源dataset的一个引用
- train: 用以分辨这些数据是否是来自训练集
#### 方法
- __init__(self, data = None, dataset = None, device = None, train = True): 从一系列的例子中生成batch


# dataset类未知，所以先看一下那个???

## torchtext.data.dataset.py
### class torch.utils.data.Dataset[source]
An abstract class representing a Dataset.<br>
All other datasets should subclass it. All subclasses should override __len__, that provides the size of the dataset, and __getitem__, supporting integer indexing in range from 0 to len(self) exclusive.
### class Dataset(torch.utils.data.Dataset)
看完还是不知道example具体是长什么样的。因为使用了getattr，所以他也是词典吗？？
#### 属性
- sort_key: 用于确定该dataset中例子的排列顺序，从而达到，当获得batch的时候，相似的例子能够被分到一个batch中，的效果
- examples: dataset中的例子
- fields: 

字典类，其中的每个项代表属性名和对应值域的一个匹配。举个例子，属性单词对应的值域就是他的词汇表。他可以作为dataset中的一个field其中的每个属性对应example的一个列也就是说，example中的每个列代表一种属性，一个example对应列的值代表这个例子该属性对应的取值。

#### 方法
- __init__(self, examples, fields, filter_pred = None)

把给定的examples和fileds，包装成dataset类。并且通过filter_pred过滤掉不符合要求的examples。这里examples是一个example的列表，fields是tuples(name, filed)的列表。filter_pred是一个过滤器，其输入是example，输出一个Boolean表达是否符合要求。<br>
(然而并不是进行提取，而只是简单的进行包装wrapping)
- @classmethod splits(cls, path, train = None, validation = None, test = None, **kwargs)

官方解释是，这个方法为数据集的多个分割建立数据集。那么理解就是，前提是不知道谁用什么方法分好了数据存储到path下面，然后这个方法的作用就是读取这些数据，分别用dataset包装一下，field等其他参数还是用原来的参数值。至于该方法中的几个参数则分别表示各个分好的数据的名字，用于分别提取。最后把生成的三个datasets打包成tuple返回。

- __getitem__(self, i): 项目索引
- __len__(self)
- __iter__(self):

for x in self.examples:
    yield x

- __getattr__(self, attr):

if attr in self.fields:
    for x in self.examples:
        yield getattr(x, attr)

### class TabularDataset(Dataset)
# 这里要用到example类的内容，需要先看完那个再回来看这个？？？？！！！

## torch.data.example.py
### class Example(object)
定义一个用于训练和测试的example：其中example的每个列代表一个属性
看不懂啊，fields里面的vals中的单个val都是一个tuple吗？？？
# 所以在这之间，已经看了field类，但是还是看不懂。？？？？

## torchtext.data.field.py
每个数据集都包含一种到多种数据。举个例子，一个文本分类的数据集就包括文本和对应分类两种数据集。在这里数据集中的每种类型的数据都分别由一个Field来进行描述。而这里每个Field，都包含一个词典，用于描述所以可能的数据以及其对应的数据编码。在生成Field的参数中，还包含一些用于描述如何生成对应编码的参数。
### 属性
见方法中的init()方法
### 方法

In [ ]:
def __init__(self, sequential = True，# 是否数据类型是序列类型的
               use_vocab = True, # 是否使用Vocab对象，如果是false的话，这个Field中的数据应该已经是numerical的
               init_token = None, # 被加到每个example前面的字符（如代表句首的符号），None代表不加任何字符
               eos_token = None, # 被加到每个example句末的字符（如代表句末的符号）， None代表不加任何字符
               fix_length = None, # 如果设定的话，每个使用这个filed的example，如果长度没有达到这个长度的话，会被padding到这个长度。
               tensor_type = torch.LongTensor, # 用于描述数据的对应的torch.Tensor类型。
               preprocessing = None, # The Pipeline that will be applied to examples using this field after tokenizing but before numericalizing
               postprocessing = None, # A Pipleline that will be applied to examples using this field after numericalizing but before the numbers are turned into a Tensor
               lower = False, # 是否把大写文本改为小写
               tokenize = (lambda s: s.split()) # 用于把string转化成序列（sequence）的函数. If "spacy", the SpaCy English tokenizer is used
               include_lengths = False # 判断，在返回minibatch的时候，是否要加上每个example的长度信息。
               batch_first = False # Whether to produce tensors with the batch dimension first？？？有何区别？？？
               pad_token = '<pad>' # 用于padding的符号
              )

- preprocessing(self, x): 对输入x进行预处理
- pad(self, minibatch): 对minibatch中的数据进行padding，如果self.fix_length存在则到这个数，否则则到给定数据中最长的那个数的长度。
- build_vocab(self, *args, **kwargs): 根据dataset中的数据（或其他可循环的数据），为这个field创建vocab。如果使用了dataset，则所有与这个field相关的column，都会被用于产生这个vocab

for arg in args:
    if isinstance(arg, Dataset):
        source += [getattr(arg, name) for name, field in arg.fields.items() if field is self]
    else: sources.append(arg)
- numericalize(self, arr, device = None, trina = True): 把使用这个fields的batch中的example，转化为Variable类
# Vocab类还没有看，应该看一下了？？？

## torchtext.data.pipeline.py
定义一个用于转换序列数据的工作流。
The input is assumed to be utf-8 encoded `str` (Python 3) or `unicode` (Python 2).
### 属性
- convert_token: 将用于处理输入序列数据的方法, 如果为None，那么就会直接使用identity function（return a copy of input）
- pipes: 被按照顺序用于输入序列的工作流

### 方法
- __init__(self, convert_token = None)
- __call__(self, x *args): 把当前的pipeline（s）应用于输入x
- call(self, x, *args): 仅把convert_token应用于输入，如果输入是一个list，那将作用于list中的每一个实例
- add_before(self, pipeline): 在当前的pipeline之前增加一个pipeline，参数pipeline可以是一个Pipeline，也可以是一个可调用的方法。
- add_after(self,pipeline): 在当前pipeline之后增加一个pipeline，参数pipeline可以是一个Pipeline，也可以是一个可调用的方法
- identity(x): 单纯复制输入x，并进行输出


## torchtext.vocab.py
### class Vocab(object),
定义一个词汇表对象，他将被用于数字化field
#### 属性
- freqs: 是一个collections.Counter对象，记录每个标价在数据中出现的次数
- stoi: 是一个collections.defaultdict实例，mapping token strings to numerical identifiers.
- itos: a list of token strings indexed by their numerical identifiers.
#### 方法
- __init__(self, counter, max_size = None, min_freq = 1, specials = ['<pad>'], vectors = None):
# 为完待续。。。